In [ ]:
pyspark --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.2

In [ ]:
from pyspark.sql.functions import *
from pyspark.sql.types import *


In [ ]:
sc.addFile("/home/adminuser/mymodel/emotion0628.py")
import emotion0628

In [ ]:
sc.addFile("/home/adminuser/mymodel/abuse0628.py")
import abuse0628

In [ ]:
df_raw = spark \
.readStream \
.format('kafka') \
.option('kafka.bootstrap.servers', 'localhost:9092') \
.option("startingOffsets", "earliest") \
.option('subscribe', 'inputchat') \
.load()


In [ ]:
json_data = df_raw.selectExpr('CAST(value AS STRING) as value')

In [ ]:
schema = StructType([ \
StructField("video_id",StringType(),True), \
StructField("user_name",StringType(),True), \
StructField("chat_id",StringType(),True), \
StructField("chat_text",StringType(),True), \
StructField("h_token", ArrayType(StringType()),True), \
StructField("modified_text",StringType(),True), \
StructField("noun_token",ArrayType(StringType(),True)), \
StructField("chat_time", TimestampType(),True)
])

In [ ]:
df_data = json_data.select(from_json(col("value"), schema).alias('data'))

In [ ]:
df_data.withColumn("emotion",lit(emotion0628.sentiment_predict("data.h_token"))) \
.withColumn("abuse",lit(abuse0628.slang_predict("data.h_token"))) \
.selectExpr("CAST(data AS STRING) AS key", "to_json(struct(*)) AS value") \
.writeStream    \
.format('kafka') \
.option('kafka.bootstrap.servers', 'localhost:9092') \
.option('topic', 'outchat') \
.option("truncate", False).option("checkpointLocation", "/tmp/dtn/checkpoint") \
.start().awaitTermination()